# Imports

In [3]:
# !CFLAGS="-I/usr/include" LDFLAGS="-L/usr/lib -lenet" pip install pyenet

In [4]:
# !pip install git+https://github.com/vladfi1/libmelee.git

In [5]:
# !apt-get update
# !apt-get install -y cmake libenet-dev

In [6]:
# !pip install peppi_py

In [7]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
import melee
import numpy as np
import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, ConcatDataset

# Data Processing (Deprecated)

In [8]:
os.getcwd()

NameError: name 'os' is not defined

In [9]:
# console = melee.Console(is_dolphin=False,
#                         allow_old_version=False,
#                         path="./data/BTSSmash-Game_20200126T202356.slp"
#                         )
# console.connect()

# while True:
#     gamestate = console.step()
#     # step() returns None when the file ends
#     if gamestate is None:
#         break
#     print("Frame " + str(gamestate.frame))
#     for _, player in gamestate.players.items():
#         print("\t", player.stock, player.percent)

In [10]:
# import peppi_py as peppi
# import numpy as np

# def get_numpy_row(pre, post):
#     def safe_extract(val):
#         return val.to_numpy() # if val is not None else np.nan

#     features = []

#     # --- Pre ---
#     features += [
#         safe_extract(pre.position.x),
#         safe_extract(pre.position.y),
#         safe_extract(pre.joystick.x),
#         safe_extract(pre.joystick.y),
#         safe_extract(pre.triggers),
#         safe_extract(pre.buttons),
#         safe_extract(pre.buttons_physical),
#         safe_extract(pre.direction),
#         safe_extract(pre.percent) if pre.percent else np.nan
#     ]

#     # --- Post ---
#     features += [
#         safe_extract(post.position.x),
#         safe_extract(post.position.y),
#         safe_extract(post.percent),
#         safe_extract(post.shield),
#         safe_extract(post.combo_count),
#         safe_extract(post.state),
#         safe_extract(post.airborne) if post.airborne else np.nan,
#         safe_extract(post.stocks),
#         safe_extract(post.last_hit_by),
#         safe_extract(post.character),
#         safe_extract(post.direction),
#     ]

#     return features

# def slp_to_numpy(path, num_ports=2):
#     game = peppi.read_slippi(path)
#     rows = []

#     for port in range(num_ports):
#         frame = game.frames
#         try:
#             pre = frame.ports[port].leader.pre
#             post = frame.ports[port].leader.post
#             row = get_numpy_row(pre, post)
#             rows.append(row)
#         except Exception:
#             # If frame/port is missing or broken, skip or pad
#             rows.append([np.nan] * 20)

#     arr = np.array(rows)
#     # transpose to (12158, 2, 20)
#     arr = arr.transpose(2, 0, 1)
#     return arr.reshape(arr.shape[0], -1)

# # Example usage
# np_data = slp_to_numpy("./drive/MyDrive/icl_smash-main/data/Stream-Game_20220828T225335.slp")
# print("Shape:", np_data.shape)
# print(np_data[:5])


In [11]:
np.save("training_data.npy", np_data)

NameError: name 'np_data' is not defined

In [ ]:
# import numpy as np
# from collections import defaultdict
# import json

# ### Step 0: Load Raw Data ###
# raw_data = np.load('./drive/MyDrive/icl_smash-main/training_data/[libmelee]example1_sampled.npy', allow_pickle=True)

# ### Step 1: Flatten a Single Frame ###
# def flatten_frame(frame):
#     flat = {}
#     for i, item in enumerate(frame):
#         key = f"col_{i}"
#         if isinstance(item, dict):  # like <Button.BUTTON_A: 'A'>: True
#             for k, v in item.items():
#                 new_key = f"{key}_{str(k)}"
#                 flat[new_key] = float(v) if isinstance(v, bool) else v
#         else:
#             flat[key] = item
#     return flat

# ### Step 2: Flatten Entire Dataset ###
# flat_data = [flatten_frame(frame) for frame in raw_data]

# ### Step 3: Build Vocab for Categorical/String Fields ###
# def build_vocab(flat_data):
#     vocab = defaultdict(set)
#     for row in flat_data:
#         for k, v in row.items():
#             if isinstance(v, str) or str(v) == 'nan':
#                 vocab[k].add(v)
#     return {k: {val: i for i, val in enumerate(sorted(list(vals)))} for k, vals in vocab.items()}

# vocab = build_vocab(flat_data)

# ### Step 4: Encode Each Frame to Numeric ###
# def encode_flat_row(row, vocab):
#     encoded = []
#     for k, v in row.items():
#         if k in vocab:
#             encoded.append(vocab[k].get(v, 0))
#         else:
#             try:
#                 encoded.append(float(v))
#             except:
#                 encoded.append(0.0)
#     return encoded

# encoded_data = np.array([encode_flat_row(row, vocab) for row in flat_data], dtype=np.float32)
# print(f"Encoded data shape: {encoded_data.shape}")  # Should be (842, <num_features>)

# ### Step 5: Sliding Window to Create Sequences ###
# def create_sequences(data, input_len=10, target_len=5):
#     input_seqs = []
#     target_seqs = []
#     for t in range(len(data) - input_len - target_len): # TODO: Currently slides 1 timestep at a time, but if we are going to infer M of them, can have the inferred M be the last M frames in the next window input
#         input_seqs.append(data[t : t + input_len]) # say, frames 10-19, t=10
#         target_seqs.append(data[t + input_len : t + input_len + target_len]) # say, frames 20-24, target_len = 5
#     return np.array(input_seqs), np.array(target_seqs)

# input_len = 10
# target_len = 5
# input_seqs, target_seqs = create_sequences(encoded_data, input_len, target_len)

# print("Input shape:", input_seqs.shape)     # (num_samples, 10, num_features)
# print("Target shape:", target_seqs.shape)   # (num_samples, 5, num_features)

# ### Step 6 (Optional): Save Output ###
# np.save("melee_input_seqs.npy", input_seqs)
# np.save("melee_target_seqs.npy", target_seqs)

# # Save vocab so you can decode predictions later
# with open("melee_vocab.json", "w") as f:
#     json.dump({k: dict(v) for k, v in vocab.items()}, f)

# print("✅ Preprocessing complete.")


# EDA

In [9]:
import os
import melee
import numpy as np
import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, ConcatDataset
import pandas as pd
import importlib
import data_processing
importlib.reload(data_processing)
from data_processing import main
main("./data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211.slp", "./old_data", "viewable")

(637, 59)

In [16]:
# Load viewable data and columns
viewable_data = np.load("./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz", allow_pickle=True)['inputs'][:, 0, :]

with open('./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_columns.txt', 'r') as f:
    text = f.read()
columns = text.split("\n")[:-1]

# Create DataFrame
df = pd.DataFrame(viewable_data.astype('float32'), columns=columns)

# # Display basic information
print("DataFrame shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

DataFrame shape: (623, 59)

Columns: ['0: frame', '1: distance', '2: p1_action_frame', '3: p1_controller_c_stick_x', '4: p1_controller_c_stick_y', '5: p1_controller_l_shoulder (has a range)', '6: p1_controller_r_shoulder (has a range)', '7: p1_controller_main_stick_x', '8: p1_controller_main_stick_y', '9: p1_button_button_a', '10: p1_button_button_b', '11: p1_button_button_x', '12: p1_button_button_y', '13: p1_button_button_z', '14: p1_button_button_l (0 or 1)', '15: p1_button_button_r (0 or 1)', '16: p1_button_button_start', '17: p1_facing', '18: p1_hitlag_left', '19: p1_hitstun_frames_left', '20: p1_jumps_left', '21: p1_off_stage', '22: p1_on_ground', '23: p1_percent', '24: p1_position_x', '25: p1_position_y', '26: p1_shield_strength', '27: p1_stock', '28: p3_action_frame', '29: p3_controller_c_stick_x', '30: p3_controller_c_stick_y', '31: p3_controller_l_shoulder', '32: p3_controller_r_shoulder', '33: p3_controller_main_stick_x', '34: p3_controller_main_stick_y', '35: p3_button_butt

In [23]:
for column in df.columns:
    print(column, sum(df[column] == 0)/df.shape[0] , "\n")

0: frame 0.0 

1: distance 1.0 

2: p1_action_frame 0.016051364365971106 

3: p1_controller_c_stick_x 0.0032102728731942215 

4: p1_controller_c_stick_y 0.0016051364365971107 

5: p1_controller_l_shoulder (has a range) 0.7865168539325843 

6: p1_controller_r_shoulder (has a range) 0.7865168539325843 

7: p1_controller_main_stick_x 0.0032102728731942215 

8: p1_controller_main_stick_y 0.0016051364365971107 

9: p1_button_button_a 0.9695024077046549 

10: p1_button_button_b 0.9662921348314607 

11: p1_button_button_x 1.0 

12: p1_button_button_y 0.9357945425361156 

13: p1_button_button_z 0.9983948635634029 

14: p1_button_button_l (0 or 1) 0.8426966292134831 

15: p1_button_button_r (0 or 1) 1.0 

16: p1_button_button_start 1.0 

17: p1_facing 0.47351524879614765 

18: p1_hitlag_left 1.0 

19: p1_hitstun_frames_left 0.7158908507223114 

20: p1_jumps_left 0.11556982343499198 

21: p1_off_stage 0.8940609951845907 

22: p1_on_ground 0.5040128410914928 

23: p1_percent 0.13162118780096307 


# Define Constants and Parameters

In [4]:
DATA_FOLDER_PATH = "./training_data"
NUM_ENUM_COLUMNS = 5  # Last 5 columns of the numpy arrays

# Data Loading and Model Training

In [1]:
import importlib
import data_loading
import models
importlib.reload(data_loading) 
importlib.reload(models)
from models import MeleeEncoderDecoder, EnumEmbeddingModule 
from data_loading import BatchedFilesDataset, MeleeDataset
import pandas as pd
from torch.utils.data import DataLoader

In [2]:
# Load the data
data = np.load("./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz", allow_pickle=True)
inputs = data['inputs']

# Find columns with non-numeric data
non_numeric_cols = []
for col in columns:
    if not np.issubdtype(df[col].dtype, np.number):
        print(f"Column '{col}' has non-numeric type: {df[col].dtype}")
        print(f"Sample values: {df[col].head()}\n")
        non_numeric_cols.append(col)

print(f"\nTotal non-numeric columns found: {len(non_numeric_cols)}")
print(f"Non-numeric column names: {non_numeric_cols}")

NameError: name 'np' is not defined

In [43]:
dataset = MeleeDataset(data_path="./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz", match_id=0, num_enums=5)

From tensor: torch.Size([623, 10, 5])
From tensor: torch.Size([623, 5, 5])


In [49]:
batched_files = BatchedFilesDataset(DATA_FOLDER_PATH, files_per_batch=1)

batched_files.load_next_batch()
dataloader = DataLoader(batched_files.load_next_batch(), batch_size=64, shuffle=True)

From tensor: torch.Size([690, 10, 5])
From tensor: torch.Size([690, 5, 5])
Loaded ./training_data/(...) Falco vs Falco (BABE) [DL] Game_20200122T213501_sequences.npz
From tensor: torch.Size([623, 10, 5])
From tensor: torch.Size([623, 5, 5])
Loaded ./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz


In [50]:
# model = MeleeEncoderDecoder(feature_dim=dataset.inputs.shape[-1])
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# loss_fn = nn.MSELoss()

# for epoch in range(10):
#     model.train()
#     total_loss = 0

#     for x_batch, y_batch in dataloader: # frames 10-19, and 20-24
#         decoder_input = y_batch[:, :-1, :]  # teacher forcing; grabs everything up until the last frame of the sequence, 20-23
#         decoder_target = y_batch[:, 1:, :]  # grabs everything after the 1st frame, 21-24

#         preds = model(x_batch, decoder_input) # given frames 10-19 and the decoder input of 20-23, output is which frames?

#         loss = loss_fn(preds, decoder_target)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import wandb
from pathlib import Path
import os
import math

# Define enum dimensions (vocabulary sizes for each categorical feature)
enum_dims = {
    'stage': STAGES_VOCAB_AMOUNT,           # Number of possible stages
    'p1_action': ACTIONS_VOCAB_AMOUNT,      # Number of possible actions
    'p1_character': CHARACTERS_VOCAB_AMOUNT,    # Number of possible characters
    'p2_action': ACTIONS_VOCAB_AMOUNT,      # Number of possible actions
    'p2_character': CHARACTERS_VOCAB_AMOUNT     # Number of possible characters
}

# Define embedding dimensions for each enum feature
embedding_dims = {
    'stage': 16,          # Embedding dimension for stages
    'p1_action': 64,      # Embedding dimension for actions
    'p1_character': 16,   # Embedding dimension for characters
    'p2_action': 64,      # Embedding dimension for actions
    'p2_character': 16    # Embedding dimension for characters
}

def train_model(
    data_folder: str,
    model_save_path: str,
    batch_size: int = 32,
    num_epochs: int = 20,
    learning_rate: float = 1e-4,
    files_per_batch: int = 1, # FIXME: currently, multiple files per batch will error due torch.stack requiring the dimension being stacked by to be the same -> will require padding (max seq_length in the batch). This also means masking will need to be added to the Transformer encoder + decoder too.
    num_enums: int = 5,
    d_model: int = 128,
    nhead: int = 4,
    num_layers: int = 3,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
    total_files: int = 10000000000
):
    # Initialize dataset
    dataset = BatchedFilesDataset(
        data_folder=data_folder,
        files_per_batch=files_per_batch,
    )
    print(len(dataset.all_files))

    # Get first batch to determine dimensionss
    first_batch = dataset.load_next_batch()
    # continuous_dim = first_batch[0]['continuous_inputs'].shape[-1] - num_enums  # Update if needed
    continuous_dim = 59 - num_enums  # Update if needed

    # Rest of the code remains the same...
    # Initialize model
    model = MeleeEncoderDecoder(
        continuous_dim=continuous_dim,
        enum_dims=enum_dims,
        embedding_dims=embedding_dims,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer, mode='min', factor=0.5, patience=2
    # )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.8, patience=5 * min(total_files, dataset.total_files)
    )
    loss_fn = nn.MSELoss()

    # Initialize wandb
    wandb.init(
        project="melee_prediction_all",
        config={
            "learning_rate": learning_rate,
            "epochs": num_epochs,
            "batch_size": batch_size,
            "continuous_dim": continuous_dim,
            "enum_dims": enum_dims,
            "embedding_dims": embedding_dims,
            "d_model": d_model,
            "nhead": nhead,
            "num_layers": num_layers,
            "model_type": model.__class__.__name__
        }
    )
    wandb.watch(model, log="all", log_freq=100)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        batch_count = 0

        # Initialize dataset again per epoch
        dataset = BatchedFilesDataset(
            data_folder=data_folder,
            files_per_batch=files_per_batch,
        )

        # Get first batch to determine dimensionss
        first_batch = dataset.load_next_batch()

        for i in range(min(total_files, dataset.total_files)):
            batch_dataset = dataset.load_next_batch()

            continuous_inputs_list = []
            continuous_targets_list = []
            enum_inputs_dict = {name: [] for name in enum_dims.keys()}
            enum_targets_dict = {name: [] for name in enum_dims.keys()}

            for batch in batch_dataset:
                # Collect continuous data
                continuous_inputs_list.append(batch['continuous_inputs']) # Each "batch" for this is of shape (10, num_continuous_features)
                continuous_targets_list.append(batch['continuous_targets'])

                # Collect enum data
                for name in enum_dims.keys():
                    enum_inputs_dict[name].append(batch[name])
                    enum_targets_dict[name].append(batch[f'target_{name}']) # Each "batch" for this is of shape (460, 10) ... should just be 10...

            # print("enum_inputs_dict", enum_inputs_dict['stage'][2].shape)
            # Stack all tensors
            x_continuous = torch.stack(continuous_inputs_list, dim=0).to(device)
            y_continuous = torch.stack(continuous_targets_list, dim=0).to(device)

            # Standardize per batch (along batch + sequence, leaving just per feature)
            m = x_continuous.mean(dim=(0,1), keepdim=True)
            s = x_continuous.std(dim=(0,1), keepdim=True) + 1e-6
            x_continuous = (x_continuous - m) / s
            y_continuous = (y_continuous - m) / s

            x_enums = {name: torch.clamp(
                                torch.stack(tensors, dim=0),
                                min=0,
                                max=enum_dims[name] - 1).to(device)
                    for name, tensors in enum_inputs_dict.items()}
            y_enums = {name: torch.clamp(
                                torch.stack(tensors, dim=0),
                                min=0,
                                max=enum_dims[name] - 1).to(device)
                    for name, tensors in enum_targets_dict.items()}

            # Teacher forcing setup
            decoder_continuous_input = y_continuous[:, :-1, :]
            decoder_continuous_target = y_continuous[:, 1:, :]

            decoder_enum_input = {name: tensor[:, :-1] for name, tensor in y_enums.items()} # As it is a 2D array, of size (num_seq, seq_length)
            decoder_enum_target = {name: tensor[:, 1:] for name, tensor in y_enums.items()}

            #decoder_enum_input = {name: tensor[:, :-1, :] for name, tensor in y_enums.items()}
            #decoder_enum_target = {name: tensor[:, 1:, :] for name, tensor in y_enums.items()}

            # Forward pass
            continuous_pred, enum_preds = model(
                x_continuous,
                x_enums,
                decoder_continuous_input,
                decoder_enum_input
            )

            # Use model's compute_loss method
            total_batch_loss, loss_components = model.compute_loss(
                cont_preds=continuous_pred,
                enum_preds=enum_preds,
                cont_targets=decoder_continuous_target,
                enum_targets=decoder_enum_target
            )

            # Backward pass
            optimizer.zero_grad()
            total_batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += total_batch_loss.item()
            batch_count += 1
            avg_loss = total_loss / batch_count

            # Logging
            log_dict = {
                "batch_loss": total_batch_loss.item(),
                "continuous_loss": loss_components["continuous"].item(),
                "avg_loss": avg_loss,
                "learning_rate": optimizer.param_groups[0]["lr"]

            }
            # Add individual enum losses to logging
            for name, loss in loss_components.items():
                if name != "continuous":
                    log_dict[f"{name}_loss"] = loss.item()

            wandb.log(log_dict)

            scheduler.step(avg_loss)

            # Save checkpoints
            if (epoch + 1) % 5 == 0:
                checkpoint_path = Path(model_save_path) / f"checkpoint_epoch_{epoch + 1}.pt"
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': avg_loss,
                }, checkpoint_path)

    # Save final model
    torch.save(model.state_dict(), os.path.join(model_save_path, "melee_predictor_all.pt"))
    wandb.finish()

In [52]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the Wandb API key
wandb_api_key = os.environ.get("WANDB_API_KEY")
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [53]:
MODEL_SAVE_PATH = "/content/drive/MyDrive/icl_smash-main/"#"/content/drive/MyDrive/Berkeley Classes/sp25/CS182/icl_smash-main/melee_predictor.pt"

# Training
train_model(
    data_folder=DATA_FOLDER_PATH,
    model_save_path=MODEL_SAVE_PATH,
    batch_size=32,
    num_epochs=20
)

/Users/alexoon/Desktop/smash_bot/smash_venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


From tensor: torch.Size([690, 10, 5])
From tensor: torch.Size([690, 5, 5])
Loaded ./training_data/(...) Falco vs Falco (BABE) [DL] Game_20200122T213501_sequences.npz


From tensor: torch.Size([623, 10, 5])
From tensor: torch.Size([623, 5, 5])
Loaded ./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz


RuntimeError: mat1 and mat2 shapes cannot be multiplied (6230x54 and 225x128)

In [ ]:
# import torch

# # suppose you still have y_mean, y_std from training,
# # both of shape (feature_dim,)
# # and you moved them to the same device as your model:

# device    = next(model.parameters()).device
# y_mean    = y_mean.to(device)    # shape (feature_dim,)
# y_std     = y_std.to(device)     # shape (feature_dim,)

# model.eval()
# with torch.no_grad():
#     # grab one test input
#     input_seq = dataset.inputs[-1].unsqueeze(0).to(device)  # (1, in_len, feat_dim)
#     # start decoder with one “zero” frame
#     decoder_input = torch.zeros((1, 1, dataset.inputs.shape[-1]),
#                                 device=device)

#     preds_norm = []
#     for _ in range(5):
#         out = model(input_seq, decoder_input)     # (1, cur_tgt_len, feat_dim)
#         next_step = out[:, -1:, :]                # (1, 1, feat_dim)
#         preds_norm.append(next_step)
#         decoder_input = torch.cat([decoder_input, next_step], dim=1)

#     preds_norm = torch.cat(preds_norm, dim=1)     # (1, 5, feat_dim)

#     # --- Undo the scaling ---
#     # reshape mean/std to broadcast over (batch, time, feature)
#     mu = y_mean.view(1, 1, -1)    # (1,1,feat_dim)
#     sigma = y_std.view(1, 1, -1)  # (1,1,feat_dim)

#     preds_denorm = preds_norm * sigma + mu      # (1, 5, feat_dim)

#     # now preds_denorm is in the original units
#     # you can convert to NumPy if you like:
#     preds_denorm = preds_denorm.squeeze(0).cpu().numpy()  # (5, feat_dim)

# print(preds_denorm.shape)  # e.g. (5, feat_dim)


(5, 119)
